# TOpic Model
## Read data

In [1]:
import pandas as pd
import numpy as np

In [47]:
df = pd.read_csv('../data/processed_data.csv')

In [48]:
df.head()

,hmid,wid,reflection_period,original_hm,cleaned_hm,modified,num_sentence,ground_truth_category,predicted_category,text
0,27673,2053,24h,i went on a successful date with someone i fel...,i went on a successful date with someone i fel...,True,1,NaN,affection,went successful date someone felt sympathy con...
1,27674,2,24h,i was happy when my son got 90% marks in his e...,i was happy when my son got 90% marks in his e...,True,1,NaN,affection,happy son got mark examination
2,27675,1936,24h,i went to the gym this morning and did yoga.,i went to the gym this morning and did yoga.,True,1,NaN,exercise,went gym morning yoga
3,27676,206,24h,we had a serious talk with some friends of our...,we had a serious talk with some friends of our...,True,2,bonding,bonding,serious talk friend flaky lately understood go...
4,27677,6227,24h,i went with grandchildren to butterfly display...,i went with grandchildren to butterfly display...,True,1,NaN,affection,went grandchild butterfly display crohn conser...


In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100535 entries, 0 to 100534
Data columns (total 10 columns):
hmid                     100535 non-null int64
wid                      100535 non-null int64
reflection_period        100535 non-null object
original_hm              100535 non-null object
cleaned_hm               100535 non-null object
modified                 100535 non-null bool
num_sentence             100535 non-null int64
ground_truth_category    14125 non-null object
predicted_category       100535 non-null object
text                     100534 non-null object
dtypes: bool(1), int64(3), object(6)
memory usage: 7.0+ MB


## Vectorization

Do this using the bag-of-words model:

- Count how many times does a word occur in each message (Known as term frequency)

- Weigh the counts, so that frequent tokens get lower weight (inverse document frequency)

- Normalize the vectors to unit length, to abstract from the original text length (L2 norm)



In [50]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [71]:
df = df[df['text'].isnull() == False]

In [79]:
bow_transformer = CountVectorizer().fit(df['text'])
# Print total number of vocab words
print(len(bow_transformer.vocabulary_))

24907


In [80]:
text_bow = bow_transformer.transform(df['text'])

## LDA

In [81]:
from sklearn.decomposition import LatentDirichletAllocation

In [84]:
n_components = 10
lda = LatentDirichletAllocation(n_components=n_components, max_iter=50,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)

In [85]:
lda.fit(text_bow)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=50.0,
             max_doc_update_iter=100, max_iter=50, mean_change_tol=0.001,
             n_components=10, n_jobs=None, n_topics=None, perp_tol=0.1,
             random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [86]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

In [89]:
n_top_words = 10

In [90]:
feature_names = bow_transformer.get_feature_names()
print_top_words(lda, feature_names, n_top_words)

Topic #0:
found dog day went took im trip watching around child
Topic #1:
time family went first friend happy moment life month long
Topic #2:
favorite one lunch food college spent didnt delicious girlfriend good
Topic #3:
happy made day got last work job birthday feel event
Topic #4:
happiness thought completed looking song state lost coming person right
Topic #5:
able go work day got gave get morning show ive
Topic #6:
wanted better someone summer date purchased bike ever actually computer
Topic #7:
friend new game bought daughter school house old went mother
Topic #8:
new got work month received finished finally ate working week
Topic #9:
car money got able seeing book sleep brought get company



In [93]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda, text_bow, bow_transformer)

/Users/wangxinquan/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.153419  0.072380       1        1  19.706933
1     -0.167981 -0.205832       2        1  14.011477
7     -0.154461 -0.237905       3        1  11.480362
0     -0.079358  0.028443       4        1   9.869815
2      0.028483  0.128651       5        1   8.877805
8     -0.067603  0.015381       6        1   8.396750
5     -0.038034  0.216370       7        1   8.286079
9      0.057880  0.210975       8        1   8.043058
4      0.258484 -0.132619       9        1   6.077889
6      0.316009 -0.095845      10        1   5.249833, topic_info=      Category          Freq         Term         Total  loglift  logprob
9747   Default  17557.000000        happy  17557.000000  30.0000  30.0000
14658  Default   8608.000000          new   8608.000000  29.0000  29.0000
8527   Default  10264.000000       friend  10264.000000  28.0000  28.0000
12984  Default  10731.000000         made  10731.000000  27.0000  27.0000
22269  Default   9228.000000         time   9228.000000  26.0000  26.0000
9223   Default  12897.000000          got  12897.000000  25.0000  25.0000
34     Default   5045.000000         able   5045.000000  24.0000  24.0000
24510  Default   7704.000000         work   7704.000000  23.0000  23.0000
7667   Default   4479.000000       family   4479.000000  22.0000  22.0000
8044   Default   4371.000000        first   4371.000000  21.0000  21.0000
8402   Default   3626.000000        found   3626.000000  20.0000  20.0000
14079  Default   5058.000000        month   5058.000000  19.0000  19.0000
5398   Default   9636.000000          day   9636.000000  18.0000  18.0000
24179  Default   9178.000000         went   9178.000000  17.0000  17.0000
7781   Default   2871.000000     favorite   2871.000000  16.0000  16.0000
8773   Default   2985.000000         game   2985.000000  15.0000  15.0000
2548   Default   2813.000000       bought   2813.000000  14.0000  14.0000
6222   Default   2528.000000          dog   2528.000000  13.0000  13.0000
9106   Default   2953.000000           go   2953.000000  12.0000  12.0000
14010  Default   3279.000000       moment   3279.000000  11.0000  11.0000
17691  Default   2175.000000     received   2175.000000  10.0000  10.0000
3206   Default   2112.000000          car   2112.000000   9.0000   9.0000
12483  Default   3238.000000         life   3238.000000   8.0000   8.0000
5378   Default   3227.000000     daughter   3227.000000   7.0000   7.0000
15183  Default   5256.000000          one   5256.000000   6.0000   6.0000
19125  Default   2311.000000       school   2311.000000   5.0000   5.0000
11598  Default   3065.000000          job   3065.000000   4.0000   4.0000
10432  Default   2178.000000        house   2178.000000   3.0000   3.0000
12731  Default   3118.000000         long   3118.000000   2.0000   2.0000
14046  Default   1792.000000        money   1792.000000   1.0000   1.0000
...        ...           ...          ...           ...      ...      ...
7264   Topic10    642.813994         ever    643.696183   2.9456  -4.3011
207    Topic10    614.809377     actually    615.691576   2.9455  -4.3456
4411   Topic10    612.520284     computer    613.402418   2.9455  -4.3493
16901  Topic10    409.039406        price    409.921567   2.9448  -4.7531
17634  Topic10    391.448061     realized    392.330253   2.9447  -4.7971
21352  Topic10    353.627247       sunday    354.509417   2.9445  -4.8987
4170   Topic10    341.416089    colleague    342.298206   2.9444  -4.9338
9935   Topic10    337.631636        heart    338.513851   2.9444  -4.9450
15369  Topic10    326.556792       others    327.439265   2.9443  -4.9783
10233  Topic10    290.424822      holiday    291.306993   2.9439  -5.0956
15266  Topic10    289.194767  opportunity    290.076956   2.9439  -5.0998
5193   Topic10    265.177672          cup    266.059804   2.9437  -5.1865
22606  Topic10    260.766344     training    261.6484